In [2]:
"""Cats vs. Dogs classification"""
import cv2
import numpy as np
from sklearn import svm
import glob
import csv
import pickle


# Get image paths and the corresponding labels for the Training set.
# Return: list of path str, list of label str
def getTrainImg():
    image_paths = []
    image_labels = []
    Y_Train = None
    
    # Read lines from the Y_Train with label and name info, into dict format
    with open('Y_Train.csv', 'r') as f:
        reader = csv.DictReader(f)
        Y_Train = list(reader)

    # extract path and labels
    for image in Y_Train:
        # folder + name = path
        image_paths.append('X_Train/'+image['Image'])
        image_labels.append(image['Label'])
        
    print('Number of images: ', len(image_paths))
    print('Sample data:', image_paths[:5],image_labels[:5])

    return image_paths, image_labels

# Calculate descriptors for each image, and save the descriptor list into 'descriptors.p'.
# Input: list of str
# Return: void
def getDes(image_paths):
    descriptors = []
    for path in image_paths:
        im = cv2.imread(path)
        _, des = sift.detectAndCompute(im,None)
        descriptors.append(des)
    pickle.dump(descriptors, open("descriptors.p", "wb"))
    print('Descriptors computed!')

# Compute the corresponding feature in the BoW vocabulary for the image in path.
# Input: BoW extractor, str of image path
# Return: list of features
def getImageData(bow_extract, path):
    im = cv2.imread(path)
    features = bow_extract.compute(im, sift.detect(im))
    return features

# Train the linearSVC by kmeans bag of words.
# input: list of label str, list of path str
# return: void
def train(image_labels, image_paths):
    # Load precalculated descriptor for all training images
    descriptors = pickle.load(open("descriptors.p", "rb"))
    # Initialize Brute Force Matcher
    matcher = cv2.BFMatcher()
    # Initialize Bag of Words descriptor extractor
    bow_extract  = cv2.BOWImgDescriptorExtractor(sift,matcher)
    # Initialize bag of words k-means trainer, with 50 clusters
    bow_train = cv2.BOWKMeansTrainer(50)
    # Add all training descriptors into the bow k-means trainer
    for des in descriptors:
        bow_train.add(des)
        
    # Cluster the descriptors into 50 vocabularies
    voc = bow_train.cluster()
    # Set these vocabularies to the BoW descriptor extractor
    bow_extract.setVocabulary(voc)
    
    # Describe each training image with the 50 clusted vocabularies
    traindata = []
    for path in image_paths:
        features = getImageData(bow_extract, path)
        traindata.extend(features)
    
    # Initialize Linear Support Vector Classification
    clf = svm.LinearSVC()
    # Train the LinearSVC classifier by fitting the image words according to their labels
    clf.fit(traindata, np.array(image_labels))
    # Save the trained linearSVC classifier and the vocabularies into 'vocabulary.p'
    pickle.dump((voc,clf), open("vocabulary.p", "wb"))
    print('Training completed!')


# Predicting the test set using the trained linearSVC
# return: list of predicted labels
def predict():
    # Load the vocabulary and the trained linearSVC classifier
    voc,clf = pickle.load(open("vocabulary.p", "rb"))
    # Initialize brute force matcher
    matcher = cv2.BFMatcher()
    # initialize the bag of word descriptor extractor
    bow_extract = cv2.BOWImgDescriptorExtractor(sift, matcher)
    # set BoW extractor vocabulary using the previously loaded vocabulary
    bow_extract.setVocabulary(voc)

    # get a list of all pathes under the X_Test folder
    test_paths = glob.glob('X_Test/*.jpg')
    
    test_results = []
    for path in test_paths:
        # Describe each test image using the BoW extractor with the 50 clusted vocabularies
        features = getImageData(bow_extract, path)
        # Predict which of the two labels should each image belong to, using linearSVC classifier
        prediction = clf.predict(features)
        # put every predicted result into a list
        test_results.append(prediction)  
    
    # write the predicted results into 'Y_Test.csv'
    with open('Y_Test.csv', 'w') as csvfile:
        fieldnames = ['Image', 'Label']
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
        
        writer.writeheader()
        for path, label in zip(test_paths, test_results):
            # the path[7:] is used to remove the 'X_Test/' folder name from the image path
            writer.writerow({'Image': path[7:], 'Label': label[0]})
    
    # The result value is only for display purpose
    return test_results


"""main"""
# Initiate SIFT detector with 100 top feature threshold
sift = cv2.xfeatures2d.SIFT_create(100)

# Note: the following three steps can be executed seperately
'''1. calculate descriptors'''
image_paths, image_labels = getTrainImg()
getDes(image_paths) # a temporary data will be saved in 'descriptor.p' 

'''2. Train'''
image_paths, image_labels = getTrainImg()
train(image_labels, image_paths) # a temporary data will be saved in 'vocabulary.p' 

'''3. Predict''' # The complete prediction result is saved in 'Y_Test.csv'
print('Sample Predict Results: ', predict()[:10])

Number of images:  5907
Sample data: ['X_Train/0.jpg', 'X_Train/1.jpg', 'X_Train/2.jpg', 'X_Train/3.jpg', 'X_Train/4.jpg'] ['1', '1', '0', '0', '0']
Descriptors computed!
Training completed!
Sample Predict Results:  [array(['1'], 
      dtype='<U1'), array(['1'], 
      dtype='<U1'), array(['1'], 
      dtype='<U1'), array(['1'], 
      dtype='<U1'), array(['1'], 
      dtype='<U1'), array(['1'], 
      dtype='<U1'), array(['1'], 
      dtype='<U1'), array(['0'], 
      dtype='<U1'), array(['0'], 
      dtype='<U1'), array(['1'], 
      dtype='<U1')]


In [20]:
"""Rearrange result file"""
#This Block is used to arrange the result data in the 'Y_Test.csv' into asending order by file name
import csv

# read the result file with name and label
Y_Test = None
with open('Y_Test.csv', 'r') as f:
    reader = csv.DictReader(f)
    Y_Test = list(reader)

for Y in Y_Test:
    # remove '.jpg' and cast to int
    Y['Image'] = int(Y['Image'][:-4])

# sort by key = 'image'
Y_Test.sort(key=lambda item:item['Image'])

for Y in Y_Test:
    # cast back to str and add back '.jpg'
    Y['Image'] = str(Y['Image']) + '.jpg'

# write the arranged result back into the file Y_Test.csv
with open('Y_Test.csv', 'w') as csvfile:
    fieldnames = ['Image', 'Label']
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)

    writer.writeheader()
    writer.writerows(Y_Test)

print('Arranged sample output: ',Y_Test[:5])

[{'Image': '0.jpg', 'Label': '1'}, {'Image': '1.jpg', 'Label': '1'}, {'Image': '2.jpg', 'Label': '1'}, {'Image': '3.jpg', 'Label': '1'}, {'Image': '4.jpg', 'Label': '0'}]
